## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-08-18-56-53 +0000,nyt,China Is Investigating Meta’s Acquisition of t...,https://www.nytimes.com/2026/01/08/business/me...
1,2026-01-08-18-54-15 +0000,nyt,Live Updates: Senate Advances Measure to Curb ...,https://www.nytimes.com/live/2026/01/08/world/...
2,2026-01-08-18-52-26 +0000,nypost,Megyn Kelly mocks Tony Dokoupil for crying on ...,https://nypost.com/2026/01/08/media/megyn-kell...
3,2026-01-08-18-51-28 +0000,cbc,Conservative MP calls cancellation of his camp...,https://www.cbc.ca/news/politics/garnett-geniu...
4,2026-01-08-18-51-19 +0000,nyt,Venezuela Leaders to Free Political Prisoners ...,https://www.nytimes.com/2026/01/08/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,83
17,venezuela,38
57,ice,31
58,shooting,28
56,minneapolis,22


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
393,2026-01-07-19-57-13 +0000,cbc,As Trump administration doubles down after ICE...,https://www.cbc.ca/news/world/livestory/ice-sh...,180
78,2026-01-08-16-53-34 +0000,nyt,We Pressed Trump on His Conclusion About the I...,https://www.nytimes.com/2026/01/08/us/politics...,180
49,2026-01-08-17-36-48 +0000,nyt,Videos Contradict Trump Administration Account...,https://www.nytimes.com/video/us/1000000106310...,176
301,2026-01-08-00-40-05 +0000,nyt,U.S. Details Plan for Venezuela’s Oil Sales Af...,https://www.nytimes.com/2026/01/07/business/en...,169
68,2026-01-08-17-08-00 +0000,wsj,The Senate voted Thursday to advance a motion ...,https://www.wsj.com/politics/policy/senate-vot...,164


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
393,180,2026-01-07-19-57-13 +0000,cbc,As Trump administration doubles down after ICE...,https://www.cbc.ca/news/world/livestory/ice-sh...
389,89,2026-01-07-20-06-00 +0000,wsj,Push for New Canadian Oil Pipeline Gets Boost ...,https://www.wsj.com/world/americas/push-for-ne...
79,49,2026-01-08-16-53-06 +0000,nypost,ICE agent who opened fire on Renee Good was dr...,https://nypost.com/2026/01/08/us-news/ice-agen...
156,49,2026-01-08-13-00-00 +0000,wsj,The U.S. pumps more oil than any other country...,https://www.wsj.com/business/energy-oil/us-ven...
261,48,2026-01-08-04-25-00 +0000,wsj,"Rep. Steny Hoyer of Maryland, a longtime membe...",https://www.wsj.com/politics/policy/steny-hoye...
142,43,2026-01-08-14-00-00 +0000,latimes,Newsom moves to reshape who runs California's ...,https://www.latimes.com/california/story/2026-...
167,41,2026-01-08-12-01-00 +0000,wsj,China Warns AI Startups Seeking to Emulate Met...,https://www.wsj.com/tech/china-warns-ai-startu...
124,37,2026-01-08-14-56-31 +0000,nypost,‘CBS Evening News’ No. 2 producer fired during...,https://nypost.com/2026/01/08/media/cbs-evenin...
75,37,2026-01-08-16-59-34 +0000,bbc,Saudi-led coalition says UAE helped smuggle se...,https://www.bbc.com/news/articles/c5y2mvp3r2do...
207,35,2026-01-08-10-30-00 +0000,wsj,Some mail won’t be postmarked the day you drop...,https://www.wsj.com/politics/policy/a-letter-w...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
